In [7]:
import pandas as pd
import numpy as np
import time
import importlib.machinery
import sys
sys.path.append('/home/sac086/extrasensory/')
import extrasense as es
from sklearn.metrics import accuracy_score, make_scorer, roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression

In [2]:
import xgboost as xgb

# Load data

In [3]:
features_df = es.get_impersonal_data(leave_users_out=[], data_type="activity", labeled_only=True)

timestamps = features_df.pop('timestamp')
label_source = features_df.pop("label_source")
labels = features_df.pop("label")
user_ids = features_df.pop("user_id")

In [4]:
for l in labels.unique():
    print(l)

SITTING
FIX_walking
OR_standing
LYING_DOWN
FIX_running
BICYCLING
STAIRS


In [6]:
folds = es.get_uids_from_es_folds()

In [8]:
def get_baseline_model():
    steps = []
    steps.append(('standardize', StandardScaler()))
    steps.append(('clf', LogisticRegression(class_weight='balanced')))
    model = Pipeline(steps)
    return model

In [24]:
def get_train_test_ind(test_fold_uids, all_user_ids):
    bool_arr = all_user_ids.isin(test_fold_uids)
    test_ind = all_user_ids.index[bool_arr]
    bool_arr = np.logical_not(bool_arr)
    train_ind = all_user_ids.index[bool_arr]
    return train_ind, test_ind
    

In [25]:
train_ind, test_ind = get_train_test_ind(folds[0], user_ids)

In [26]:
def get_metrics(y, y_pred, verbose=True):
    predictions = []
    # Naive accuracy (correct classification rate):
    accuracy = np.mean(y_pred == y);
    
    # Count occorrences of true-positive, true-negative, false-positive, and false-negative:
    tp = np.sum(np.logical_and(y_pred,y));
    tn = np.sum(np.logical_and(np.logical_not(y_pred),np.logical_not(y)));
    fp = np.sum(np.logical_and(y_pred,np.logical_not(y)));
    fn = np.sum(np.logical_and(np.logical_not(y_pred),y));
    
    # Sensitivity (=recall=true positive rate) and Specificity (=true negative rate):
    sensitivity = float(tp) / (tp+fn);
    specificity = float(tn) / (tn+fp);
    
    # Balanced accuracy is a more fair replacement for the naive accuracy:
    balanced_accuracy = (sensitivity + specificity) / 2.;
    
    # Precision:
    # Beware from this metric, since it may be too sensitive to rare labels.
    # In the ExtraSensory Dataset, there is large skew among the positive and negative classes,
    # and for each label the pos/neg ratio is different.
    # This can cause undesirable and misleading results when averaging precision across different labels.
    precision = float(tp) / (tp+fp);
    
    if verbose:
        print("-"*10);
        print('Accuracy*:         %.2f' % accuracy);
        print('Sensitivity (TPR): %.2f' % sensitivity);
        print('Specificity (TNR): %.2f' % specificity);
        print('Balanced accuracy: %.2f' % balanced_accuracy);
        print('Precision**:       %.2f' % precision);
        print("-"*10);
        
    return {'sensitivity' : sensitivity,
            'specificity' : specificity,
            'accuracy' : accuracy,
            'balanced accuracy' : balanced_accuracy,
            'precision' : precision}

In [29]:
def test_model(model_getter, context):
    fold_metrics = []
    
    for i, kf in enumerate(folds):
        print('Fold #%s' % i)
        model = model_getter()
        
        train_ind, test_ind = get_train_test_ind(kf, user_ids)
        
        print("Training model...")
        X_train = features_df.iloc[train_ind]
        y_train = labels.iloc[train_ind]
        y_train = np.array([1 if y == context else 0 for y in y_train])
        
        X_test = features_df.iloc[test_ind]
        y_test = labels.iloc[test_ind]
        y_test = np.array([1 if y == context else 0 for y in y_test])

        model.fit(X_train, y_train)
        
        print("Testing model...")
        y_pred = model.predict(X_test)
        
        metrics = get_metrics(y_test, y_pred, verbose=True)
        fold_metrics.append(metrics)
    
    return fold_metrics

In [32]:
test_metrics = test_model(get_baseline_model, 'SITTING')

Fold #0
Training model...
Testing model...
----------
Accuracy*:         0.65
Sensitivity (TPR): 0.54
Specificity (TNR): 0.74
Balanced accuracy: 0.64
Precision**:       0.64
----------
Fold #1
Training model...
Testing model...
----------
Accuracy*:         0.60
Sensitivity (TPR): 0.49
Specificity (TNR): 0.68
Balanced accuracy: 0.59
Precision**:       0.56
----------
Fold #2
Training model...
Testing model...
----------
Accuracy*:         0.65
Sensitivity (TPR): 0.55
Specificity (TNR): 0.73
Balanced accuracy: 0.64
Precision**:       0.61
----------
Fold #3
Training model...
Testing model...
----------
Accuracy*:         0.68
Sensitivity (TPR): 0.56
Specificity (TNR): 0.75
Balanced accuracy: 0.66
Precision**:       0.58
----------
Fold #4
Training model...
Testing model...
----------
Accuracy*:         0.59
Sensitivity (TPR): 0.56
Specificity (TNR): 0.62
Balanced accuracy: 0.59
Precision**:       0.61
----------


In [42]:
def get_mean_metrics(metrics, verbose=True):
    mean_metrics = {}
    
    for fold_metrics in metrics:
        for key, val in fold_metrics.items():
            if key in mean_metrics:
                mean_metrics[key].append(val)
            else:
                mean_metrics[key] = [val]
    
    print("-"*10);
    print('Accuracy*:         %.2f' % np.mean(mean_metrics['accuracy']));
    print('Sensitivity (TPR): %.2f' % np.mean(mean_metrics['sensitivity']));
    print('Specificity (TNR): %.2f' % np.mean(mean_metrics['specificity']))
    print('Balanced accuracy: %.2f' % np.mean(mean_metrics['balanced accuracy']))
    print('Precision**:       %.2f' % np.mean(mean_metrics['precision']))
    print("-"*10);

In [43]:
get_mean_metrics(test_metrics)

----------
Accuracy*:         0.63
Sensitivity (TPR): 0.54
Specificity (TNR): 0.70
Balanced accuracy: 0.62
Precision**:       0.60
----------


In [44]:
from collections import Counter
c = Counter(labels)
for key, val in c.items():
    print("%s : %s" % (key, val))

OR_standing : 37622
STAIRS : 822
FIX_running : 1090
BICYCLING : 5020
LYING_DOWN : 104208
FIX_walking : 22135
SITTING : 136346


In [45]:
def get_xgb_model():
    steps = []
    steps.append(('standardize', StandardScaler()))
    steps.append(('clf', xgb.XGBClassifier()))
    model = Pipeline(steps)
    return model

In [46]:
test_metrics = test_model(get_xgb_model, 'SITTING')

Fold #0
Training model...
Testing model...
----------
Accuracy*:         0.67
Sensitivity (TPR): 0.64
Specificity (TNR): 0.69
Balanced accuracy: 0.66
Precision**:       0.64
----------
Fold #1
Training model...
Testing model...
----------
Accuracy*:         0.65
Sensitivity (TPR): 0.57
Specificity (TNR): 0.72
Balanced accuracy: 0.64
Precision**:       0.62
----------
Fold #2
Training model...
Testing model...
----------
Accuracy*:         0.71
Sensitivity (TPR): 0.59
Specificity (TNR): 0.81
Balanced accuracy: 0.70
Precision**:       0.71
----------
Fold #3
Training model...
Testing model...
----------
Accuracy*:         0.68
Sensitivity (TPR): 0.66
Specificity (TNR): 0.70
Balanced accuracy: 0.68
Precision**:       0.58
----------
Fold #4
Training model...
Testing model...
----------
Accuracy*:         0.59
Sensitivity (TPR): 0.54
Specificity (TNR): 0.64
Balanced accuracy: 0.59
Precision**:       0.62
----------


In [47]:
get_mean_metrics(test_metrics)

----------
Accuracy*:         0.66
Sensitivity (TPR): 0.60
Specificity (TNR): 0.71
Balanced accuracy: 0.66
Precision**:       0.63
----------
